In [16]:
import cv2
import time
import os
from pathlib import Path

import concurrent
import requests

In [17]:
def get_meta(img: cv2.Mat):
    image_data = cv2.imencode('.jpg', img)[1].tobytes()
    urls = [
        
        "https://72c4-34-143-252-179.ngrok.io/useColor",
        "https://52c7-35-247-181-246.ngrok.io/predict"
    ]

    def request_post(url, data):
        return requests.post(url, files={"image": data}).json()

    with concurrent.futures.ThreadPoolExecutor() as executor: # optimally defined number of threads
        res = [executor.submit(request_post, url, image_data) for url in urls]
        concurrent.futures.wait(res)

    return {"color": res[0].result()["predicted"], "model": res[1].result()["predicted"]}

In [8]:
def crop_do (coor, img, conf, cls):
    current_time = int(round(time.time() * 1000))
    print(current_time)
    if not os.path.exists("./runs/"):
        os.makedirs("./runs/")
    Path("./runs/Use1/").mkdir(parents=True, exist_ok=True)
    crop_img = img[ int(coor[1]) : int(coor[3]), int(coor[0]): int(coor[2])]
    #result = cv2.resize(crop_img, (640, 640))
    #result = cv2.cvtColor(result , cv2.COLOR_RGB2GRAY)
    meta = get_meta(crop_img)
    print(meta)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, f'Color: {meta["color"]}', (10,80), font, 3, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(img, f'Model: {meta["model"]}', (10,160), font, 3, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imwrite("./runs/Use1/img" + str(current_time) + ".jpg", img)

In [19]:
from lean_detect import UseModel
veh = UseModel("yolov7.pt", detect_class=[2, 5, 7], confident=0.75)

YOLOR 🚀 2022-12-7 torch 1.12.1 CPU



Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
Model Summary: 306 layers, 36905341 parameters, 36905341 gradients
/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:482.)
  return self._grad


 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



In [10]:
veh.detect("./clips/use001.MP4" , do_function=crop_do)

1676910856195
{'color': 'Black', 'model': 'Mazda2'}
1676910878825
{'color': 'White', 'model': 'yaris_old'}
1676910890839
{'color': 'Grey', 'model': 'unknown'}
1676910917750
{'color': 'Bronze', 'model': 'e150'}
1676910956913
{'color': 'Black', 'model': 'unknown'}
1676910972707
{'color': 'Blue', 'model': 'unknown'}
1676910984259
{'color': 'Brown', 'model': 'acv70'}
1676911044339
{'color': 'Black', 'model': 'yaris_mama'}
1676911066751
{'color': 'Black', 'model': 'civic_fc'}
1676911078366
{'color': 'Black', 'model': 'unknown'}
1676911153672
{'color': 'Blue', 'model': 'e160'}
1676911188745
{'color': 'White', 'model': 'rt50'}
1676911225722
{'color': 'Grey', 'model': 'unknown'}
1676911246312
{'color': 'Grey', 'model': 'unknown'}
1676911267000
{'color': 'Grey', 'model': 'unknown'}
1676911281245
{'color': 'Black', 'model': 'Mazda2'}
1676911288581
{'color': 'Black', 'model': 'unknown'}
1676911293145
{'color': 'Black', 'model': 'unknown'}
1676911295670
{'color': 'Brown', 'model': 'unknown'}
16769

# Google storage

In [5]:
! pip install google-cloud
! pip install pymongo

  Using cached google_cloud-0.34.0-py2.py3-none-any.whl (1.8 kB)


In [2]:
from google.cloud import storage
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r'./carrgclassification-857e3c375cdd.json'

def upload_cs_file(bucket_name, source_file_name, destination_file_name): 
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(destination_file_name)
    blob.upload_from_filename(source_file_name)

In [7]:
import datetime

def get_cs_file_url(bucket_name, file_name, expire_in=datetime.datetime.now() + datetime.timedelta(4000)): 
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    url = bucket.blob(file_name).generate_signed_url(expire_in)

    return url

In [3]:
upload_cs_file('images-bucks', "./DemoCase/img1676910856195.jpg", "test.jpg")

In [10]:
url = get_cs_file_url('images-bucks', "test.jpg")
url

'https://storage.googleapis.com/images-bucks/test.jpg?Expires=2022666100&GoogleAccessId=image-buck-sa%40carrgclassification.iam.gserviceaccount.com&Signature=cvXJubE%2BArPl2Z%2FrgmTN8pIU8w54gBJgVjd2roEXdUrm%2B2sbWpjVj%2FPF5R%2FFlXepsrQk4umm1gJ2Fm2qWi8nHW8XGyXoijH24%2F4p70ouiIj9UrS4M9twTlR0dEpb6w%2FAEyTWZK9yU2irtFpVClltdWgDA4TCTt8YBVSg5g%2BbRqaX5y3wlPRV16fdfqaZGzEVrYHtSn%2B8V%2FuCR2097ymAYBpOuSOYTl1hP6e07EprNFqC4ALQ3vP%2B3HFvXTVkC0ZziviE54m%2FIa1SaHUuSgPyYpCxahOAbYNBR6mcbsuo57oLQ9fEqhugVzjENkohyEEmXWcR9wkYORLDiRRUFHokjQ%3D%3D'

In [20]:
import database as db

In [21]:
import json
 
# Opening JSON file
with open('car_meta.json') as json_file:
    car_meta = json.load(json_file)

In [23]:
def cloud_image (bucket_name, source_file_name, destination_file_name):
    upload_cs_file(bucket_name, source_file_name, destination_file_name)
    url = get_cs_file_url(bucket_name, destination_file_name)
    return url


In [24]:
def crop_do (coor, img, conf, cls):
    current_time = int(round(time.time() * 1000))
    print(current_time)
    if not os.path.exists("./runs/"):
        os.makedirs("./runs/")
    Path("./runs/Use1/").mkdir(parents=True, exist_ok=True)
    crop_img = img[ int(coor[1]) : int(coor[3]), int(coor[0]): int(coor[2])]
    #result = cv2.resize(crop_img, (640, 640))
    #result = cv2.cvtColor(result , cv2.COLOR_RGB2GRAY)
    meta = get_meta(crop_img)
    print(meta)
    car_detail = car_meta[meta["model"]]
    print(car_detail)
    cv2.imwrite(f"./runs/Use1/{current_time}.jpg", img)
    origin_img_path = cloud_image('images-bucks', f"./runs/Use1/{current_time}.jpg", f'{current_time}-origin.jpg')
    cv2.imwrite(f"./runs/Use1/{current_time}-crop.jpg", crop_img)
    crop_img_path = cloud_image('images-bucks', f"./runs/Use1/{current_time}-crop.jpg", f'{current_time}-crop.jpg')

    db.update_data(car_detail["make"], car_detail["model"], car_detail["class"], "Unknown", meta["color"], origin_img_path, None, crop_img_path)
    

In [22]:
veh.detect("./clips/use001.MP4" , do_function=crop_do)

Exception: ERROR: /Users/thisisthanick/Library/Mobile Documents/com~apple~CloudDocs/Dev/CarDetection_WebApp/clips/use001.MP4 does not exist

In [1]:
from detect_flow import detect_flow

YOLOR  e4c417a torch 1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191.375MB)



Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


c:\Users\PPUNN\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model Summary: 306 layers, 36905341 parameters, 36905341 gradients, 104.5 GFLOPS
c:\Users\PPUNN\anaconda3\lib\site-packages\torch\nn\modules\module.py:561: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  if param.grad is not None:


 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



In [2]:
detect_flow("./clips/use001.MP4")

1677695255777
{'color': 'Black', 'model': 'Mazda2', 'possible': [{'class': 'Mazda2', 'conf': 0.7645111680030823}, {'class': 'corolla_cross', 'conf': 0.023793384432792664}, {'class': 'acv40', 'conf': 0.022760875523090363}, {'class': 'e160b', 'conf': 0.020936021581292152}, {'class': 'e150', 'conf': 0.019416043534874916}], 'plate_num': '7กล838', 'plate_url': 'https://storage.googleapis.com/images-bucks/16776952590673-plate.jpg?Expires=2023201118&GoogleAccessId=image-buck-sa%40carrgclassification.iam.gserviceaccount.com&Signature=V9BR9V38xKpbH4gogz9iumA4vPQZmQw0G1fAWjuVF6XCSyPzOdeqHgXXXK6DppByLXwlZQjmkpAeNY0KoDvzWUqGhTAqHPinyEonLPX5D304G9zz7OC8moZtBjppkfjNpO7PAt5x1KmecRlsYjzFc%2BkwdpB1I%2BoN18MKVq2GUr9Vnz3AxW1dM%2BCmYvcs%2FL8XYvSmAvSB0JCFPGFZ1VIypVDE9kl47pKv3hFqfg9l48P2%2F%2FU32djR3YrHT9p1GPGqe2SItAS9pqXauUa76eBFWTmC2qE7ZUWnG0v1WlU4aq8kIWyIczKmvaQNdfyVXU4Ozned%2FRLLPswJlqvgSSV9zg%3D%3D'}
{'make': 'Mazda', 'model': 'Mazda 2', 'class': 'Eco'}
1677695283814
{'color': 'White', 'model': 'yaris_

In [1]:
! python app.py

^C
